# Data Extraction
This notebook is for reading in the BookIt data and Absenteeism data. It also includes calculation to assume the home seating location of employees in the BookIt dataset. 

## Imports

In [1]:
import pandas as pd
import numpy as np
import glob

## Read in BookIt & Absenteeism Data

In [2]:
def read_multi_excel(path):
    '''
    Given a file path with wildcard for extension, parse all files with that extension in directory 
    into a single dataframe.
    '''
    
    all_files = glob.glob(path)
    li = []
    
    for filename in all_files:
        df = pd.read_excel(filename, index_col=None, header=1)
        li.append(df)
        
    df = pd.concat(li, axis=0, ignore_index=True)
    
    return df

In [3]:
BookIt_df = read_multi_excel("data/bookit/*.xls")

WARNING *** file size (163712) not 512 + multiple of sector size (512)
WARNING *** file size (139136) not 512 + multiple of sector size (512)


In [4]:
absence_df = pd.read_excel("data/absenteeism/OGPLeaveData.xlsx")

## Convert columns with timestamp values to the datetime format for calculations. 

In [5]:
def date_convert(df, columns):
    '''
    Given a dataframe and columns, convert all columns to the indicated datetime format.
    '''
    
    for col in columns:
        df[col]= pd.to_datetime(df[col], format ='%m/%d/%Y %I:%M %p', errors='coerce')
    
    return df

In [6]:
dt_columns = ['Start Time',
          'End Time',
          'Active Start',
          'Active End',
          'Check In',
          'Check Out',
          'Created',
          'Setup Time',
          'Break Down Time',
          'Max Cancel Time',
          'Bumpable Time']

In [7]:
BookIt_df = date_convert(BookIt_df, dt_columns)

## Filter to only 1800 F Street Facility, Workspace reservation type and remove null values for Active End to reflect only completed reservations.

In [8]:
BookIt_df = BookIt_df[BookIt_df['Facility']=='1800 F Street']
BookIt_df = BookIt_df[BookIt_df['Category']=='Workspace']
BookIt_df = BookIt_df[BookIt_df['Active End'].notnull()]

## Calculate the number of business days of the reservation. For values with 0 days but any increment of time greater than 0, reflect a reservation of 1 day. 

In [9]:
def days_calc(df, calc_col, start, end):
    '''
    Given a dataframe, the new column for the calculated values, start column and end column, calculate 
    the number business days between the start column and end column, writing that value to calc_col.
    For values less than 1 but greater than 0, use the value 1 to capture partial days as a full day.
    '''
    df[calc_col] = df.apply(lambda x:
                                                1 if np.busday_count(x[start].date(), x[end].date()) == 0
                                                and x[start] < x[end]
                                                else np.busday_count(x[start].date(), x[end].date()), axis= 1)
    
    return df

In [10]:
BookIt_df = days_calc(BookIt_df,'Reservation Days','Active Start','Active End')

## Format For User column to merge with Absenteeism Data. 

In [11]:
BookIt_df['For User'] = BookIt_df['For User'].str.split(', ').str[::-1].str.join(' ').str.strip()

## Capture the aggregated amount of days booked per Floor/Wing per employee.

In [12]:
summary_df = BookIt_df[['For User', 'Facility Floor','Reservation Days']].groupby(['For User','Facility Floor'], 
                                                                     sort=False)['Reservation Days'].max().reset_index(name='Reservation Days')

## Capture the most booked Floor/Wing for each employee.

In [13]:
summary_df = summary_df.groupby('For User').apply(lambda x: x['Facility Floor'][x['Reservation Days'].idxmax()]).reset_index(name='Reservation Days')

## Merge the two datsets so that the Absenteeism data is supplemented with most requently booked Floor/Wing for users if it is User is within BookIt data.

In [14]:
merged_df = pd.merge(summary_df, absence_df, left_on=['For User'], right_on=['Name'], how='left')